In [77]:
import pandas as pd

In [78]:
data = pd.read_csv('~/Downloads/Coinbase_BTCUSD_d.csv')

In [79]:
low_memory = True

In [80]:
#Data Cleaning & Manipulation

In [81]:
#drop rows where volume is zero due to inactivity in coinbase's earlier days
data.drop(data.tail(56).index,inplace=True)

In [82]:
#calculate bitcoin volatility for a single day 
price_fluc = []
for index,row in data.iterrows():   
    try:
        cur_high = float(row['High'])
    except ValueError,e:
        row['High'] = -1
    try:
        cur_low = float(row['Low'])
    except ValueError,e:
        row['Low'] = -1
    if cur_low == -1 or cur_high == -1:
        cur_fluc = -1
    else:
        cur_fluc = cur_high - cur_low
    price_fluc.append(cur_fluc)

In [83]:
data['price fluctuation'] = price_fluc

In [84]:
data['price fluctuation'].max()

6150.6500000000015

In [85]:
data['price fluctuation'].argmax()

/Users/tiffanymckenzie/Virtualenvs/tiffany_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


517

In [86]:
data.iloc[517]

Date                   12/7/17
Symbol                  BTCUSD
Open                     14090
High                   19650.7
Low                      13500
Close                    17390
Volume BTC             72755.2
Volume USD           1.154e+09
price fluctuation      6150.65
Name: 517, dtype: object

In [ ]:
#NOW WE WILL START TO ADD IN RELEVANT FEATURES NECESSARY TO START SPECULATION CALCULATION

In [87]:
#add market cap and velocity as a columns/features
approx_market_cap = []
vel = []
for index,row in data.iterrows():   
    try:
        cur_vol = float(row['Volume BTC'])
    except ValueError,e:
        row['Volume BTC'] = -1
    try:
        cur_year = row['Date'][-2:]
    except ValueError,e:
        row['Date'] = '-1'
    if cur_year == '14':
        cur_cap = 12215255
    elif cur_year == '15':
        cur_cap = 13678725
    elif cur_year == '16':
        cur_cap = 15062925
    elif cur_year == '17':
        cur_cap = 16081388
    elif cur_year == '18':
        cur_cap = 16782650
    elif cur_year == '19':
        cur_cap = 17459488
    else:
        cur_cap = -1
    cur_vel = cur_vol / cur_cap
    approx_market_cap.append(cur_cap)
    vel.append(cur_vel)
data['velocity'] = vel
data['approx market cap'] = approx_market_cap

In [88]:
#add average price as a column/feature
avg_price = []
for index, row in data.iterrows():   
    try:
        cur_high = float(row['High'])
    except ValueError,e:
        row['High'] = -1
    try:
        cur_low = float(row['Low'])
    except ValueError,e:
        row['Low'] = -1
    if cur_low == -1 or cur_high == -1:
        cur_avg_price = -1
    else:
        cur_avg_price = cur_low + ((cur_high - cur_low)/2)
    avg_price.append(cur_avg_price)
data['average price'] = avg_price

In [89]:
#add change in volume as a column/feature
num_rows = data.shape[0]
v_list = []
for i in range(0, num_rows):
    if i == 0:
        cur_delta_v = 0
    else:
        vol1 = data.iloc[i - 1]['Volume BTC']
        vol2 = data.iloc[i]['Volume BTC']
        cur_delta_v = vol2 - vol1
    v_list.append(cur_delta_v)
data['delta vol'] = v_list

In [ ]:
#Pre-processing - calculating speculation

In [ ]:
#Approach to calculating speculation:
#Step 1. Find day where the change in velocity between current day and previous day is approximately zero 
#(we do this by finding the minimum velocity change)
#Step 2. On this day where change in velocity from previous day is zero, calculate the change in price from the previous day.
#This change in price will represent the change in the amount of speculation = (gamma)(delta_k), where delta_k is 
#initilized/assumed to be 1. From this we find constant, gamma. 
#Step 3. Since velocity is inversely proportional to speculation, we know that the maximum change in velocity, corresponds to 
#a maximum change in speculative k. If delta_v is postive, this implies that delta_k should be negative. The absolute
#maximum change in k that corresponds to this is 2, and therefore the corresponding delta_k is -2. With this, we use
#the maximum change in velocity, the maximum negative change in k and gamma, to calculate a constant,c.
#Step 4. for each row in the dataframe we use the formula delta_k = c / (delta_v * gamma) to calulate delta_k
#Step 5. We initilize the oldest row of data to speculation of 0, and iterate through the dataframe calculating
#speculative k for each day. 
#Step 6. We then multiply speculative k by gamma to get the unnormalized speculation amount
#Step 7. Then normalize the speculation amount between -1 and 1

In [90]:
#Step 1: 
min_delta_v = float('inf')
for i in range(num_rows - 1, 0, -1):
    v_one = data.iloc[i].velocity
    v_two = data.iloc[i - 1].velocity
    abs_delta_v = abs(v_two - v_one)
    if abs_delta_v == 0.0:
        min_delta_v = abs_delta_v
        min_delta_v_index = i - 1
        break
    if abs_delta_v < min_delta_v:
        min_delta_v = abs_delta_v
        min_delta_v_index = i - 1
print 'min_delta_v is:', min_delta_v
print 'row index:', min_delta_v_index

min_delta_v is: 3.396364698065712e-08
row index: 398


In [91]:
#Step 2:
delta_k = 1
p_two = data.iloc[min_delta_v_index]['average price'] 
p_one  = data.iloc[min_delta_v_index + 1]['average price'] 
delta_p = p_two - p_one 
gamma = delta_p / delta_k
gamma

-318.1349999999993

In [92]:
#Step 3:
max_delta_v_index = data['delta vol'].argmax()
max__neg_delta_k = -2
constant  = data.iloc[max_delta_v_index]['delta vol'] * max__neg_delta_k * gamma
print'constant: ', constant

constant:  95451112.98359978


/Users/tiffanymckenzie/Virtualenvs/tiffany_env/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


In [93]:
#Step 4:
delta_k_list = [0]
for i in range(num_rows - 1, 0, -1):
    vol2 = data.iloc[i]['Volume BTC']
    vol1 = data.iloc[i - 1]['Volume BTC']
    vol_change = vol2 - vol1
    cur_delta_k = constant / (vol_change * gamma)
    delta_k_list.append(cur_delta_k)
delta_k_list.reverse()
data['delta k'] = delta_k_list

#step 4.5 : normalize the delta k between -2 and 2
#norm_delta_k_list = []
#for i in range(0, num_rows):
 #   cur_delta_k = data.iloc[i]['delta k']
  #  min_delta_k = data['delta k'].min()
   # max_delta_k = data['delta k'].max()
    #norm_delta_k = (4 * ((cur_delta_k - min_delta_k) / (max_delta_k - min_delta_k))) - 2
    #norm_delta_k_list.append(norm_delta_k)
#data['normalized delta k'] = norm_delta_k_list
data.tail()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,price fluctuation,velocity,approx market cap,average price,delta vol,delta k
1559,1/30/15,BTCUSD,234.52,244.59,227.89,232.55,5699.69,1333424.96,16.70,0.000417,13678725,236.240,130.43,-311.868780
1560,1/29/15,BTCUSD,236.14,242.00,221.14,234.52,6661.74,1555210.37,20.86,0.000487,13678725,231.570,962.05,199.830403
1561,1/28/15,BTCUSD,262.69,265.99,228.50,236.14,5160.30,1272214.28,37.49,0.000377,13678725,247.245,-1501.44,85.412671
1562,1/27/15,BTCUSD,270.00,278.47,244.03,262.69,1647.55,432840.93,34.44,0.000120,13678725,261.250,-3512.75,-85.609101
1563,1/26/15,BTCUSD,254.53,322.72,241.43,270.00,5152.24,1439367.10,81.29,0.000377,13678725,282.075,3504.69,0.000000


In [94]:
#Step 5:
k_list = []
for i in xrange(num_rows - 1, -1, -1):  
    if i == num_rows - 1:
        cur_k = 0
    else:
        cur_delta_k = data.iloc[i]['delta k']
        cur_k = k_list[len(k_list) - 1] + cur_delta_k
    k_list.append(cur_k)
k_list.reverse()
data['k'] = k_list
data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,price fluctuation,velocity,approx market cap,average price,delta vol,delta k,k
0,5/20/19,BTCUSD,8200.00,8200.00,7570.31,8118.39,19601.59,154601032.6,629.69,0.001123,17459488,7885.155,0.00,-69.432076,-236647.527503
1,5/19/19,BTCUSD,7262.40,8315.00,7252.58,8200.00,23922.84,189749462.4,1062.42,0.001370,17459488,7783.790,4321.25,22.985936,-236578.095427
2,5/18/19,BTCUSD,7363.69,7491.22,7204.42,7262.40,10869.93,79710779.2,286.80,0.000623,17459488,7347.820,-13052.91,-9.526559,-236601.081363
3,5/17/19,BTCUSD,7878.96,7940.75,6600.00,7363.69,42364.34,306073578.9,1340.75,0.002426,17459488,7270.375,31494.41,21.880696,-236591.554804
4,5/16/19,BTCUSD,8203.32,8388.00,7660.74,7878.96,28652.10,229351414.5,727.26,0.001641,17459488,8024.370,-13712.24,73.273767,-236613.435500


In [95]:
#Step 6: 
specs = []
for i in range(0, num_rows):
    spec = data.iloc[i]['k'] * gamma
    specs.append(spec)
data['unnormalized speculation'] = specs

In [96]:
#Step 7:
norm_k_list = []
for i in range(0, num_rows):
    cur_k = data.iloc[i]['unnormalized speculation']
    min_k = data['unnormalized speculation'].min()
    max_k = data['unnormalized speculation'].max()
    norm_k = (2 * ((cur_k - min_k) / (max_k - min_k))) - 1
    norm_k_list.append(norm_k)
data['normalized speculation'] = norm_k_list
data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,price fluctuation,velocity,approx market cap,average price,delta vol,delta k,k,unnormalized speculation,normalized speculation
0,5/20/19,BTCUSD,8200.00,8200.00,7570.31,8118.39,19601.59,154601032.6,629.69,0.001123,17459488,7885.155,0.00,-69.432076,-236647.527503,7.528586e+07,0.950375
1,5/19/19,BTCUSD,7262.40,8315.00,7252.58,8200.00,23922.84,189749462.4,1062.42,0.001370,17459488,7783.790,4321.25,22.985936,-236578.095427,7.526377e+07,0.950129
2,5/18/19,BTCUSD,7363.69,7491.22,7204.42,7262.40,10869.93,79710779.2,286.80,0.000623,17459488,7347.820,-13052.91,-9.526559,-236601.081363,7.527109e+07,0.950210
3,5/17/19,BTCUSD,7878.96,7940.75,6600.00,7363.69,42364.34,306073578.9,1340.75,0.002426,17459488,7270.375,31494.41,21.880696,-236591.554804,7.526805e+07,0.950176
4,5/16/19,BTCUSD,8203.32,8388.00,7660.74,7878.96,28652.10,229351414.5,727.26,0.001641,17459488,8024.370,-13712.24,73.273767,-236613.435500,7.527502e+07,0.950254


In [97]:
data.to_csv('~/Downloads/btc_spec.csv', index=False)